# SVM Ejercicio - Hiperplano de Margen Máximo - Caso Lineal


In [2]:
import sklearn
from sklearn import datasets
from sklearn import svm
#from sklearn import metrics

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

# Conjuntos Linealmente Separables:

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html 

In [3]:
N=1000  # total de datos a generar.
noisy_data = sklearn.datasets.make_blobs(n_samples=N, n_features=2, centers=[(-4,-4),(4,4)], cluster_std=1.0, random_state=10)  # centers=2

In [ ]:
X, Y = noisy_data
Y = Y.reshape(Y.shape[0],1)
print(X.shape)
print(Y.shape)

In [ ]:
print(X[0:6,:])

In [ ]:
print(Y[0:6,0])

### Dibujemos algunas rectas que separen ambas clases:

In [ ]:
sns.set(rc={'figure.figsize':(9,7)})   # (width, height) - definiendo el tamaño de las gráficas.

In [ ]:
plt.scatter(X[:,0], X[:,1], c=Y, s=20, cmap=plt.cm.Spectral);  # Puntos de ambas clases

xhs = np.linspace(-8, 8)   # Algunas rectas-clasificadores lineales: Hiperplano/margen Separador
for m, b, d in [(-1, 0.7, 2.4),  (-0.2, -1, 0.8)]:      # (m,b,d) --> m: pendientes  /  b: ordenadas en el origen   /  d: margen máximo
    yhs = m * xhs + b
    plt.plot(xhs, yhs, '-k');
    #plt.fill_between(xhs, yhs - d, yhs + d, edgecolor='none',color='#AAAAAA', alpha=0.3);  # márgenes máximos de cada HiperplanoSeparador

#plt.plot([-2], [1], 'x', color='green',  markersize=10);  # Un nuevo punto que deseamos clasificar... ¿a qué clase la asignarías?

##**Modelo SVM:**

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html 

In [ ]:
model = svm.SVC(kernel='linear', C=1E10)
model.fit(X, Y)


### Una función para visualizar los hiperplanos separadores e identicar los vectores de soporte:

In [ ]:
def plot_svc_decision_function(model, ax=None, plot_support=True):    

    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)

    xy = np.vstack([X.ravel(), Y.ravel()]).T    # ravel convierte cada uno a 1-D.
    P = model.decision_function(xy).reshape(X.shape)    # generamos las predicciones para cada punto del grid.
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '-.'])    # para distinguir cada hiperplano
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=400, facecolors='yellow', alpha=0.3);
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

### Grafiquemos ahora los límites del Hiperplano Separador/Margen Máximo:

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=Y, s=20, cmap=plt.cm.Spectral)
plot_svc_decision_function(model);

Los vectores de soporte indicados en la imagen son los siguientes:

In [ ]:
model.support_vectors_

# Caso: conjuntos Linealmente No-Seprables:

In [ ]:
N=1000
noisy_data2 = sklearn.datasets.make_blobs(n_samples=N, n_features=2, centers=[(-4,-4),(4,4)], cluster_std=2.0, random_state=10)  

In [ ]:
X2, Y2 = noisy_data2
Y2 = Y2.reshape(Y2.shape[0],1)

In [ ]:
plt.scatter(X2[:,0], X2[:,1], c=Y2, s=40, cmap=plt.cm.Spectral); 

In [ ]:
C =  .1000

In [ ]:
modelnol = svm.SVC(kernel='linear', C=C)
modelnol.fit(X2, Y2)

In [ ]:
plt.scatter(X2[:, 0], X2[:, 1], c=Y2, s=50, cmap=plt.cm.Spectral)
plot_svc_decision_function(modelnol)

In [ ]:
modelnol.support_vectors_

### Seleccionado los mejores parámetros - C y Gamma:

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

In [ ]:
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)

In [ ]:
C_range

In [ ]:
gamma_range

In [ ]:
param_grid = dict(gamma=gamma_range, C=C_range)

In [ ]:
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

In [ ]:
grid = GridSearchCV(svm.SVC(), param_grid=param_grid, cv=cv)

In [ ]:
grid.fit(X2, Y2)

In [ ]:
print("Los mejores parametros son %s con un score de %0.2f"
      % (grid.best_params_, grid.best_score_))